In [1]:
import pymysql
import json
coon = pymysql.connect(
    host = '10.15.82.50',user = 'root',passwd = '123',
    port = 3306,db = 'lsc',charset = 'utf8'
    )
cur = coon.cursor()

In [2]:
entities = set()
cur.execute("select entity from conc3 group by entity")
tem = cur.fetchall()
for single in tem:
    entities.add(single[0])

In [3]:
def getJson(entity):
    不良反应 = []
    炮制 = []
    相关疾病 = []
    用法 = []
    功效 = []
    鉴别 = []
    
    预防 = []
    检查 = []
    
    药物 = []
    针灸 = []
    推拿 = []
    火罐 = []
    手术 = []
    偏方 = []
    
    养生 = []
    drug = [不良反应,炮制,相关疾病,用法,功效,鉴别]
    disease = [预防,检查,养生]
    sub = [药物,针灸,推拿,火罐,手术,偏方]
    cur.execute("select question, type from conc2 where entity = %s", (entity))
    tem = set()
    for line in cur.fetchall():
        tag = line[1]
        question = line[0].replace('<p>','').replace('</p>','').replace('\xa0','')
        if len(question) > 48:
            question = question[:48]+"..."
        if question in tem:
            pass
        else:
            eval(tag).append({'name': question, 'size': 3938})
            tem.add(question)

    if len(不良反应) + len(炮制) + len(相关疾病) + len(用法) + len(功效) + len(鉴别)== 0:
        #disease
        a = {'name':'药物','children':sub[0]}
        b = {'name':'针灸','children':sub[1]}
        c = {'name':'推拿','children':sub[2]}
        d = {'name':'火罐','children':sub[3]}
        e = {'name':'手术','children':sub[4]}
        f = {'name':'偏方','children':sub[5]}
        zhiliao = {'name':'治疗','children':[a,b,c,d,e,f]}
        yufang = {'name':'预防','children':disease[0]}
        jiancha = {'name':'检查','children':disease[1]}
        yangshen = {'name':'养生','children':disease[2]}
        res = {'name':entity,'children':[yufang,jiancha,zhiliao,yangshen]}
    else:
        #drug
        a = {'name':'不良反应','children':drug[0]}
        b = {'name':'炮制','children':drug[1]}
        c = {'name':'相关疾病','children':drug[2]}
        d = {'name':'用法','children':drug[3]}
        e = {'name':'功效','children':drug[4]}
        f = {'name':'鉴别','children':drug[5]}
        res = {'name':entity,'children':[a,b,c,d,e,f]}
    return json.dumps(res, ensure_ascii=False)

In [ ]:
coon2 = pymysql.connect(
    host = '10.15.82.58',user = 'root',passwd = '123',
    port = 3306,db = 'tcm',charset = 'utf8'
    )
cur2 = coon2.cursor()
query = 'insert into qa_conclusion3(name, json) values(%s, %s)'
for entit in entities:
    try:
        cur2.execute(query,(entit, getJson(entit)))
        coon2.commit()
    except:
        pass

In [6]:
# names = set()
# cur2.execute("select name from qa_conclusion2")
# tem = cur2.fetchall()
# for single in tem:
#     names.add(single[0])
# print(len(names))

7517


In [8]:
cur2.execute("select name, json from qa_conclusion")
tem = cur2.fetchall()
for single in tem:
    name = single[0]
    jso = single[1]
    if name in names:
        #update
        query = "UPDATE qa_conclusion3 SET json = %s WHERE name = %s"
        cur2.execute(query,(jso, name)) 
        coon2.commit()        
    else:
        #insert
        query = 'insert into qa_conclusion3(name, json) values(%s, %s)'
        cur2.execute(query,(name, jso)) 
        coon2.commit()